In [1]:
!pip install huggingface-hub pypdf pytest boto3 langchain unstructured chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 15.8 MB/s eta 0:00:00


In [3]:
!pip install grpcio streamlit

In [1]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import requests
from langchain_openai import ChatOpenAI

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [4]:
# Initialize the loader with the correct webpage and parameters
loader = WebBaseLoader(
    web_paths=("https://www.theguardian.com/commentisfree/2016/jun/10/amazon-fresh-culture-instant-gratification",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("article-body-commercial-selector article-body-viewer-selector dcr-fp1ya", "dcr-1bgz3hb")
        )
    ),
)

# Load the documents
docs = loader.load()

# Diagnostic step to inspect the structure of a document object
if docs:
    print("Attributes of the Document object:", dir(docs[0]))

Attributes of the Document object: ['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '_abc_impl', '_calculate_keys', '_copy_and_set_values', '_decompose_class', '_enforce_dict_if_root', '_g

In [6]:
print(docs)

[Document(page_content='In the era of instant gratification, it was perhaps inevitable that retailers would eventually offer same-day grocery delivery, and it’s hardly surprising that it’s Amazon that is leading the way. After all, you can have almost any household item delivered to your door within 24 hours of clicking “buy”; why should something as essential as food be an exception?For those of us who like the convenience of it – and who are willing to pay – it sounds revolutionary. No more battling for delivery slots from the major supermarkets; no having to plan what you want in advance. Your sudden craving for venison haunch can be satisfied that very day, and without the horror of being told off by the self-service checkout.It’s likely that take-up will be greatest among younger and more affluent shoppers, but while Amazon Fresh (which is already well-established in the US) will initially be available only in north and east London, it suggests that our collective desire for our n

In [5]:
class RecursiveCharacterTextSplitter:
    def __init__(self, chunk_size, chunk_overlap):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def split_document(self, text):
        return [text[i:i + self.chunk_size] for i in range(0, len(text), self.chunk_size - self.chunk_overlap)]

    def split_documents(self, documents):
        splits = []
        for doc in documents:
            text = doc.page_content  # Using 'page_content' attribute directly
            splits.extend(self.split_document(text))
        return splits


In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_texts(texts=splits, embedding=OpenAIEmbeddings())

In [9]:
# Set up the retriever
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

In [27]:
# Define a function to format documents for the RAG prompt
def format_strings(documents):
    # Extract 'page_content' from each Document object and join them
    return "\n\n".join(doc.page_content for doc in documents)

# rag_chain usage
formatted_context = format_strings(docs)  # Format the loaded documents
question = "What is the story about Amazon?"
rag_chain_result =  rag_chain = (
    {"context": lambda x: retriever, "question": RunnablePassthrough()} | prompt | llm
)

In [28]:
rag_chain.invoke("What is the story about Amazon?")

AIMessage(content='The story is about Amazon leading the way in offering same-day grocery delivery, emphasizing the convenience of instant gratification. Amazon Fresh, initially available in north and east London, caters to the desire for immediate fulfillment of needs. The convenience comes at a cost, contributing to a culture of instant gratification and potentially impacting values and relationships.', response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 763, 'total_tokens': 830}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-63ec4edc-1c4b-485c-a8d8-716c3aaccd89-0')